In [1]:
#!/usr/bin/env python3

import os
import sys
import threading
import numpy as np
import yaml
import pickle as pkl
import cv2
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from joblib import Parallel,delayed
# from keras.applications.mobilenet import MobileNet
# from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.


In [2]:
# All videos
all_videos = open("list/all.video").readlines()
all_videos = [i.strip() for i in all_videos]

downsampled_videos = "downsampled_videos"
CNN_FEAT_DIR = "cnn"
keyframe_interval = 20

In [3]:
def get_cnn_features_from_video(video_name,model, keyframe_interval):
    "Receives filename of downsampled video and of output path for features. Extracts features in the given keyframe_interval. Saves features in pickled file."
    # TODO
    downsampled_video_filename = os.path.join(downsampled_videos, video_name + '.mp4')
    cnn_feat_video_filename = os.path.join(CNN_FEAT_DIR, video_name + '.cnn')
    if not os.path.isfile(downsampled_video_filename):
            return
    frames = get_keyframes(downsampled_video_filename,keyframe_interval)
    features = []
    max_len = 0
    for keyframe in frames:
        img_data = image.img_to_array(keyframe)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)
        feature = model.predict(img_data)
        feature = np.array(feature)
        features.append(np.reshape(feature.flatten(),(1,-1)))

    with open(cnn_feat_video_filename,"wb") as o:
        pkl.dump(features,o)
        

def get_keyframes(downsampled_video_filename, keyframe_interval):
    "Generator function which returns the next keyframe."

    # Create video capture object
    video_cap = cv2.VideoCapture(downsampled_video_filename)
    frame = 0
    while True:
        frame += 1
        ret, img = video_cap.read()
        if ret is False:
            break
        if frame % keyframe_interval == 0:
            newimg = cv2.resize(img,(224,224))
            yield newimg
    video_cap.release()

In [4]:
with Parallel(n_jobs=4) as parallel:
    
    model = VGG16(weights='imagenet', include_top=False)
    res = parallel(delayed(get_cnn_features_from_video)(video,model,keyframe_interval) for video in tqdm(all_videos[:]))

# Without parallel threads
# for video in tqdm(all_videos[:]):
#     model = VGG16(weights='imagenet', include_top=False)
#     get_cnn_features_from_video(video,model,keyframe_interval)        

Instructions for updating:
Colocations handled automatically by placer.


  0%|          | 11/2935 [00:41<6:26:23,  7.93s/it]/home/ubuntu/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
 87%|████████▋ | 2548/2935 [10:49:51<1:08:03, 10.55s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-0eb9538ffbf4>", line 4, in <module>
    res = parallel(delayed(get_cnn_features_from_video)(video,model,keyframe_interval) for video in tqdm(all_videos[:]))
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 934, in __call__
    self.retrieve()
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/joblib/parallel.py", line 833, in retrieve
    self._output.extend(job.get(timeout=self.timeout))
  File "/home/ubuntu/anaconda3/lib/python3.7/site-packages/joblib/_parallel_backends.py", line 521, in wrap_future_result
    return future.result(timeout=timeout)
  File "/home/ubuntu/anaconda3/lib/python3.7/concurrent/futures/_base.py", line 427, in result
    self._condition.wait(timeout)
  File "/home/ubuntu/anaconda3/lib/pytho

KeyboardInterrupt: 